<a href="https://colab.research.google.com/github/deshanchathusanka/data-mining/blob/main/Associated_Rule_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Python Implementation of Association Rule Mining**

1) Brute-Force Approach

2) Apriori Approach

3) FP-Tree Algorithm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ln -s "/content/drive/My Drive/Academic/CSCM35 - Big Data & Data Mining/coursework 2/Dataset.Small" "/content/"
!ln -s "/content/drive/My Drive/Academic/CSCM35 - Big Data & Data Mining/coursework 2/Dataset.Large" "/content/"
!pip3 install line_profiler
!pip3 install memory_profiler
!pip install snakeviz
%load_ext line_profiler
%load_ext memory_profiler
%load_ext snakeviz

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ln: failed to create symbolic link '/content/Dataset.Small': File exists
ln: failed to create symbolic link '/content/Dataset.Large': File exists
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [106]:
################ import statements ##################
# %%snakeviz

from itertools import combinations
from collections import Counter
import numpy as np
import csv
import pandas as pd

In [ ]:
class FileHandler :
  """
    Operate file handling tasks 
  """
  def read_from_file(self, file_name) :
    """
      Read from CSV file 
      Input :
        file_name = File Name
    """
    csvfile = open(file = file_name, mode = 'r') # open file
    csvreader = csv.reader(csvfile, skipinitialspace = True) # csv reader object
    rows = [] # empty list
    for row in csvreader:
      rows.append(row)
    csvfile.close() #' close file
    return rows

  def write_to_file(self, file_name, data) :
    """
      Write to CSV file 
      Input :
        file_name = File Name
    """
    file = open(file = file_name, mode = 'w') # file object
    csvwriter = csv.writer(file) # csv reader object
    for row in data:
      csvwriter.writerow(row)
    file.close() # close file

In [ ]:
file_handler = FileHandler()
transactions = file_handler.read_from_file('Dataset.Small/GroceryStore.csv') # read transactions or database
file_handler.write_to_file('Transactions.csv', transactions) # write transactions or database into another file

**Brute-Force Approach(Frequent itemset mining)**

$
Number\ of\ unique\ items\ =\ d\\
Number\ of\ transactions\ =\ N\\
Average\ width\ of\ a\ transaction=\ w\\ \\
Number\ of\ all\ possible\ combinations\ =\
\displaystyle\sum_{r=1} ^{d} c_{r}\ =\ 2^d - 1\\
Complexity\ =\ O(Nw2^{d})
$

In [ ]:
def search_database(database, itemset) :
  """
    Search occurance of given itemset in the transaction database
    Inputs
      database : Transactions : List of lists 
      itemset : Itemset that should be searched : List
    Outputs
      occurance : Number of occurance : integer
  """
  # frequency = np.count_nonzero([all(item in transaction for item in itemset) for transaction in database])
  itemset_set = set(itemset)
  frequency = len([ transaction for transaction in database if itemset_set.issubset(transaction)])
  return frequency

def calc_candidate_sup_cnt(database, c_k, k) : 
  """
  Calculate support count for all candidate k-itemsets and generate dataframe
  Inputs
    database : Transactions : List of lists
    c_k : Candidate k-itemsets : List of tuples
  Output : 
    c_k_df : Candidate k-itemsets dataframe with support counts
  """
  itemset_df = pd.DataFrame(columns=['key', 'item_cnt','itemset','support_cnt'])
  for itemset in c_k: # itemset : tuple of items
      support_count = search_database(database, itemset)
      data_entry = {
        "key" : frozenset(itemset), # immutable set
        "item_cnt": k,
        "itemset": itemset,
        "support_cnt": support_count
      }
      itemset_df = itemset_df.append(data_entry, ignore_index = True)
  return itemset_df

def select_freq_itemsets(c_k, min_sup_cnt) :
  """
  Select frequent k-itemsets from candidate k-itemsets(Candidate elimination)
  Inputs
    c_k : Candidate k-itemsets : DataFrame(itemset,support_cnt)
    min_sup_cnt : minimum support count(hyper parameter) : integer
  Outputs
    f_k : Frequent k-itemsets : DataFrame(itemset,support_cnt)
  """
  f_k = c_k[c_k['support_cnt'] >= min_sup_cnt] 
  return f_k

def brute_force_approach(database, unique_items, min_sup_cnt, itemset_groups):
  """ 
  Brute Force Method 
  Inputs
    database : Transaction database : List of lists
    unique_items : Unique items in the database : List of items
    min_sup_cnt : Minimum support count : intereger
    itemset_groups : Required item groups(k values)
  Outputs
    all_freq_itemsets = Frequent itemsets for given groups = DataFrame
  """
  all_candidate_df = pd.DataFrame(columns=['item_cnt','itemset','support_cnt'])
  for k in itemset_groups: # loop number of itemset groups
    c_k = combinations(unique_items, k) # create combinations for each itemset group
    c_k_df = calc_candidate_sup_cnt(database, c_k, k)
    all_candidate_df = all_candidate_df.append(c_k_df, ignore_index = True)

  all_freq_itemsets = select_freq_itemsets(all_candidate_df, min_sup_cnt)
  return all_freq_itemsets

In [ ]:
######################### Unit Testing ::: Small dataset ###################################
transaction_df = pd.read_csv('Dataset.Small/GroceryStore.csv', header=None)

transaction_df.drop_duplicates(subset =0,
                     keep = 'first', inplace = True)
database = list(transaction_df[0].apply(lambda x:x.split(",") ))
database_set = [set(transaction) for transaction in database]
# print(f'Number of Transactions : {len(database_set)}')

unique_items = set()
for transaction in database:
  for item in transaction :
    unique_items.add(item)
print(f'Unique Items :\n {unique_items}')

min_sup_cnt = 2
itemset_groups = [1, 2, 3, 4, 5]
# %lprun -f brute_force_approach brute_force_approach(database, unique_items, min_sup_cnt, itemset_groups)
# %mprun -f brute_force_approach brute_force_approach(database, unique_items, min_sup_cnt, itemset_groups)
# %snakeviz brute_force_approach(database, unique_items, min_sup_cnt, itemset_groups)
freq_itemsets = brute_force_approach(database, unique_items, min_sup_cnt, itemset_groups)
print(f'\n\nShape of Frequent Itemsets From Brute-Force Algorithm : {freq_itemsets.shape}')
print(f'Frequent Itemsets From Brute-Force Approach(Head) \n{freq_itemsets}')


Unique Items :
 {'COCK', 'COFFEE', 'TEA', 'JAM', 'BOURNVITA', 'CORNFLAKES', 'BISCUIT', 'BREAD', 'MILK', 'MAGGI', 'SUGER'}


Shape of Frequent Itemsets From Brute-Force Algorithm : (35, 4)
Frequent Itemsets From Brute-Force Approach(Head) 
    item_cnt                 itemset support_cnt                     key
0          1                 (COCK,)           2                  (COCK)
1          1               (COFFEE,)           6                (COFFEE)
2          1                  (TEA,)           6                   (TEA)
3          1                  (JAM,)           2                   (JAM)
4          1            (BOURNVITA,)           3             (BOURNVITA)
5          1           (CORNFLAKES,)           5            (CORNFLAKES)
6          1              (BISCUIT,)           6               (BISCUIT)
7          1                (BREAD,)          11                 (BREAD)
8          1                 (MILK,)           5                  (MILK)
9          1                (MA

In [ ]:
############################# Large Datatset ####################################
transaction_df_large = pd.read_csv('Dataset.Large/OnlineRetail.csv')


####### Chean Data #########
def encode(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

def clean_data(original_dataset) :
  original_dataset['Description'] = original_dataset['Description'].str.strip()
  original_dataset.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
  original_dataset['InvoiceNo'] = original_dataset['InvoiceNo'].astype('str')
  cleaned_transaction_df = original_dataset[~original_dataset['InvoiceNo'].str.contains('C')]

  cleaned_transaction_df = (cleaned_transaction_df[cleaned_transaction_df['Country'] =="France"]
            .groupby(['InvoiceNo', 'Description'])['Quantity']
            .sum().unstack().reset_index().fillna(0)
            .set_index('InvoiceNo'))
  cleaned_transaction_df = cleaned_transaction_df.applymap(encode)
  cleaned_transaction_df.drop('POSTAGE', inplace=True, axis=1)

  return cleaned_transaction_df

transaction_df_large = clean_data(transaction_df_large)
database_large = []
unique_items_large = list(transaction_df_large.columns)
min_sup_cnt_large = 40
for index, row in transaction_df_large.iterrows():
  transaction = [item for item in unique_items_large if row[item]!=0 ]
  database_large.append(transaction)


####### Print DataFrame #########
print(f'\nLarge Transaction Database : \n{database_large}')
print(f'\nUnique Items in Large Transaction Database : \n{unique_items_large}')



Large Transaction Database : 
[['ALARM CLOCK BAKELIKE GREEN', 'ALARM CLOCK BAKELIKE PINK', 'ALARM CLOCK BAKELIKE RED', 'CHARLOTTE BAG DOLLY GIRL DESIGN', 'CIRCUS PARADE LUNCH BOX', 'INFLATABLE POLITICAL GLOBE', 'LUNCH BOX I LOVE LONDON', 'MINI JIGSAW CIRCUS PARADE', 'MINI JIGSAW SPACEBOY', 'MINI PAINT SET VINTAGE', 'PANDA AND BUNNIES STICKER SHEET', 'RED TOADSTOOL LED NIGHT LIGHT', 'ROUND SNACK BOXES SET OF4 WOODLAND', 'SET 2 TEA TOWELS I LOVE LONDON', 'SET/2 RED RETROSPOT TEA TOWELS', 'SPACEBOY LUNCH BOX', 'STARS GIFT TAPE', 'VINTAGE HEADS AND TAILS CARD GAME', 'VINTAGE SEASIDE JIGSAW PUZZLES'], ['CHARLOTTE BAG DOLLY GIRL DESIGN', 'MINI JIGSAW DOLLY GIRL', 'MINI JIGSAW SPACEBOY', 'PICTURE DOMINOES', 'POLKADOT RAIN HAT', 'VINTAGE HEADS AND TAILS CARD GAME'], ['ASSORTED COLOUR MINI CASES', 'BIG DOUGHNUT FRIDGE MAGNETS', 'EDWARDIAN PARASOL BLACK', 'EDWARDIAN PARASOL PINK', 'EDWARDIAN PARASOL RED', 'LUNCH BAG RED RETROSPOT', 'LUNCH BAG WOODLAND', 'LUNCH BOX I LOVE LONDON', 'LUNCH BOX WIT

**Apriori Algorithm(Frequent itemset mining)**

* Superset is frequent >>> Subset is frequent (Bottom Up)

* Sebset is infrequent >>> Superset is infrequent (Top down)

* This algorithm has **anti-monotone** propety >>> support of itemset can not exceed support of its subset


**Main Steps**

* Candidate Generation ($F_{k-1} >> C_{k}$)

* Candidate Pruning

* Support Counting

* Candidate Elimination ($C_{k} >> F_{k}$)

**Candidate Pruning ($F_{k-1}×F_{k-1}$)**

$Itemset\ is\ not\ frequent\ if\ one\ of\ its\ sub\ itemset\ is\ not\ frequent$

* Number of **(k-1)-size subsets** for a **k-itemset** = $C^{K}_{K-1}$ = $K$
* Number of **already verified (k-1)-size subsets** at the candidate generation = $2$
* Number of subsets for frequency verification stage per each **k-itemset** = $K-2$
* Total number of subsets for frequency verification for **all k-itemsets** = $L_{k}\times(K-2)$

In [ ]:
def generate_candidates(f_prv, k):
  """
  Generate candidate set for k-itemsets
  Input : 
    f_prv : F(K-1) 
  Output : 
    c_k : List of candidate k-itemsets : List of lists
  """
  c_k = [] # candidate k-itemset
  f_prv_pairs = combinations(f_prv, 2) # select pair of frequent (k-1)-itemsets
  for f_prv_pair in f_prv_pairs:
    if(len(f_prv_pair[0]) == 1) : ### K=2 ###
      two_itemset = [*f_prv_pair[0],*f_prv_pair[1]]
      two_itemset.sort()         
      c_k.append(two_itemset)
    elif(f_prv_pair[0][0:k-2] == f_prv_pair[1][0:k-2]) :### K>=2 and first (k-2) items are same in both itemsets ###
      generated_itemset = f_prv_pair[0][0:k-2]
      delta_items = [f_prv_pair[0][k-2],f_prv_pair[1][k-2]]
      delta_items.sort()
      generated_itemset = [*generated_itemset,*delta_items]
      c_k.append(generated_itemset)
  return c_k

def prune_candidates(c_k, f_prv, k) :
  """
  Prune candidate set from k-itemsets
  Inputs
    c_k : candidate k-itemsets : List of lists
    f_prv : frequent (k-1)-itemsets 
  Output
    c_k_prune : pruned candidate k-itemsets : List of lists
  """
  infrq_itemsets = []
  ### search candidate k-itemsets to check frequency of its (k-1)-sized subsets
  for itemset in c_k :
    subsets = combinations(itemset, k-1)
    for subset in subsets:
      frequency = search_database(f_prv, subset)
      if(frequency==0):
        infrq_itemsets.append(itemset)
        break
  ### remove infrequent itemsets from candidate k-itemsets ###
  for itemset in infrq_itemsets :
    c_k.remove(itemset)

  return c_k


In [ ]:
def apriori_algorithm(database, unique_items, itemset_groups, min_sup_cnt) :
  f_itemsets_store = {}
  all_frq_itemsets_df = pd.DataFrame(columns=['item_cnt','itemset','support_cnt'])
  for k in itemset_groups:
    if(k>1) :
      f_prv = f_itemsets_store[k-1]['itemset']
      c_k = generate_candidates(f_prv, k) # generate candidates
      c_k = prune_candidates(c_k, f_prv, k) # prune candidates
      c_k = calc_candidate_sup_cnt(database, c_k, k)
      f_k = select_freq_itemsets(c_k, min_sup_cnt)
      all_frq_itemsets_df = all_frq_itemsets_df.append(f_k, ignore_index=True)
      f_itemsets_store[k] = f_k
    else :
      c_1 = [[item] for item in unique_items] # candidate 1-itemset
      c_1 = calc_candidate_sup_cnt(database, c_1, 1)
      f_1 = select_freq_itemsets(c_1, min_sup_cnt)
      all_frq_itemsets_df = all_frq_itemsets_df.append(f_1, ignore_index=True)
      f_itemsets_store[1] = f_1
  return all_frq_itemsets_df, f_itemsets_store


In [ ]:
############################## Unit Testing ::: Small Dataset #####################################
# all_itemsets_df = %lprun -f apriori_algorithm apriori_algorithm() # execute line profiler
itemset_groups = [1, 2, 3, 4]
all_frq_itemsets_df, f_itemsets_store = apriori_algorithm(database, unique_items, itemset_groups, 2) # execute line profiler
print(f'Shape of frequent itemsets : {all_frq_itemsets_df.shape}')
print(f'Frequent itemsets from apriori approach \n{all_frq_itemsets_df.head()}')

Shape of frequent itemsets : (35, 4)
Frequent itemsets from apriori approach 
  item_cnt      itemset support_cnt          key
0        1       [COCK]           2       (COCK)
1        1     [COFFEE]           6     (COFFEE)
2        1        [TEA]           6        (TEA)
3        1        [JAM]           2        (JAM)
4        1  [BOURNVITA]           3  (BOURNVITA)


In [ ]:
############################## Unit Testing ::: Large Dataset #####################################
%lprun -f apriori_algorithm apriori_algorithm(database_large, unique_items_large, itemset_groups, 39)  # execute line profiler(lp)
itemset_groups = [1, 2, 3, 4]
all_frq_itemsets_df, f_itemsets_store = apriori_algorithm(database_large, unique_items_large, itemset_groups, 39) 
print(f'Shape of frequent itemsets : {all_frq_itemsets_df.shape}')
print(f'Frequent itemsets from apriori approach \n{all_frq_itemsets_df.head()}')

Shape of frequent itemsets : (29, 4)
Frequent itemsets from apriori approach 
  item_cnt                      itemset support_cnt  \
0        1  [ALARM CLOCK BAKELIKE PINK]          40   
1        1       [DOLLY GIRL LUNCH BOX]          39   
2        1     [LUNCH BAG APPLE DESIGN]          49   
3        1    [LUNCH BAG RED RETROSPOT]          60   
4        1  [LUNCH BAG SPACEBOY DESIGN]          47   

                           key  
0  (ALARM CLOCK BAKELIKE PINK)  
1       (DOLLY GIRL LUNCH BOX)  
2     (LUNCH BAG APPLE DESIGN)  
3    (LUNCH BAG RED RETROSPOT)  
4  (LUNCH BAG SPACEBOY DESIGN)  


**Rule Generation**

Antecedent -> Consequent

Ex : {X,Y} -> {P,Q}

In [ ]:
def generate_rules(f_k, h_m, f_itemsets_store, min_conf) :
  """
  Generate rules from frequent itemsets
  Inputs:
    f_k : Frequent k-itemsets (X,Y,P,Q) : Tuple
    h_m : Consequent (P,Q) : Tuple
    f_itemsets_store : Frequent itemset store(key = k, value = support counts) : Dictionary(Key = Integer, value = DataFrame)
  """
  conf = 0
  if(len(h_m) > 0) :
    k = len(f_k)
    m = len(h_m[0])
  else :
    return pd.DataFrame(columns = ['antecedent', 'consequent', 'confidence'])
  
  if(k > m+1) : #### Check whether an item can be passed from antecedent to consequent >>> Terminating condition of the recursive function###
    h_next = generate_candidates(h_m, m+1) # Generate (m+1) candidates for consequent
    h_next = prune_candidates(h_next, h_m, m+1) # Prune (m+1) candidates
    h_next_invalid = []
    rule_df = pd.DataFrame(columns = ['antecedent', 'consequent', 'confidence'])
    for h_next_i in h_next : # Iterate candidate consequents
      consequent = h_next_i
      antecedent = list(set(f_k) - set(h_next_i))
      rule_sup_cnt = f_itemsets_store[k].loc[f_itemsets_store[k]['key']==frozenset(f_k)]
      antecedent_sup_cnt = f_itemsets_store[len(antecedent)].loc[f_itemsets_store[len(antecedent)]['key']==frozenset(antecedent)]

      if len(rule_sup_cnt)>0 and len(antecedent_sup_cnt)>0 :
        conf = rule_sup_cnt['support_cnt'].values[0]/antecedent_sup_cnt['support_cnt'].values[0]

      if(conf > min_conf) : 
        data_entry = {
          'antecedent': antecedent,
          'consequent': consequent,
          'confidence': conf
        }
        rule_df = rule_df.append(data_entry, ignore_index = True)
      else : 
        h_next_invalid.append(h_next_i)

    for h_next_invalid_i in h_next_invalid : # Remove invalid consequents 
      h_next.remove(h_next_invalid_i)

    ### recursively generate rules for subgraphs 
    return generate_rules(f_k, h_next, f_itemsets_store, min_conf).append(rule_df, ignore_index=True)
  else :
    return pd.DataFrame(columns = ['antecedent', 'consequent', 'confidence'])

In [ ]:
def generate_rules_apriori(database, unique_items, itemset_groups, min_sup_cnt, min_conf):
  all_frq_itemsets_df, f_itemsets_store = apriori_algorithm(database, unique_items, itemset_groups, min_sup_cnt)

  rule_df = pd.DataFrame(columns = ['antecedent', 'consequent', 'confidence'])
  for f_k in all_frq_itemsets_df['itemset'] :
    h_1_invalid = []
    for item in f_k :
      if len(f_k)==1 :
        continue
      consequent = []
      consequent.append(item)
      antecedent = list(set(f_k)-set(consequent))

      rule_sup_cnt = f_itemsets_store[len(f_k)].loc[f_itemsets_store[len(f_k)]['key']==frozenset(f_k)]
      antecedent_sup_cnt = f_itemsets_store[len(antecedent)].loc[f_itemsets_store[len(antecedent)]['key']==frozenset(antecedent)]

      if len(rule_sup_cnt)>0 and len(antecedent_sup_cnt)>0 :
        conf = rule_sup_cnt['support_cnt'].values[0]/antecedent_sup_cnt['support_cnt'].values[0]

      if(conf >= min_conf) : 
        data_entry = {
          'antecedent': antecedent,
          'consequent': consequent,
          'confidence': conf
        }
        rule_df = rule_df.append(data_entry, ignore_index = True)
      else : 
        h_1_invalid.append(consequent)

    h_1 = [[item] for item in f_k] # create consequents for single items
    if len(h_1_invalid)>0 : # remove low confidence items from consequents
      for h_1_invalid_i in h_1_invalid : # Remove invalid consequents 
        h_1.remove(h_1_invalid_i)
    if len(h_1)==0 :
      continue
      
    rule_df = rule_df.append(generate_rules(f_k, h_1, f_itemsets_store, min_conf), ignore_index=True)
  
  return rule_df


In [ ]:
################################# Rule Generation : Small Datatset ########################################
min_conf = 0.5
min_sup_cnt = 3
rule_df = generate_rules_apriori(database=database, unique_items=unique_items, itemset_groups=[1,2,3,4], min_sup_cnt = min_sup_cnt, min_conf = min_conf)
print(f'Minimum Support Count : {min_sup_cnt}')
print(f'Minimum Confidence : {min_conf}')
print(f'Shape of Association Rules : {rule_df.shape}')
print(f'Association Rules(Head) \n{rule_df.head()}')

Minimum Support Count : 3
Minimum Confidence : 0.5
Shape of Association Rules : (11, 3)
Association Rules(Head) 
     antecedent    consequent  confidence
0  [CORNFLAKES]      [COFFEE]         0.6
1      [COFFEE]  [CORNFLAKES]         0.5
2       [SUGER]      [COFFEE]         0.6
3      [COFFEE]       [SUGER]         0.5
4         [TEA]       [BREAD]         0.5


In [ ]:
################################# Rule Generation : Large Datatset ########################################
min_conf = 0.5
min_sup_cnt_large = 40
%lprun -f generate_rules_apriori generate_rules_apriori(database = database_large, unique_items = unique_items_large, itemset_groups = [1,2,3,4], min_sup_cnt = min_sup_cnt_large, min_conf = min_conf)  # execute line profiler
rule_df_large = generate_rules_apriori(database = database_large, unique_items = unique_items_large, itemset_groups = [1,2,3,4], min_sup_cnt = min_sup_cnt_large, min_conf = min_conf)
print(f'Minimum Support Count : {min_sup_cnt}')
print(f'Minimum Confidence : {min_conf}')
print(f'Shape of Association Rules : {rule_df_large.shape}')
print(f'Association Rules(Head) \n{rule_df_large.head()}')

Minimum Support Count : 3
Minimum Confidence : 0.5
Shape of Association Rules : (10, 3)
Association Rules(Head) 
                           antecedent                            consequent  \
0  [PLASTERS IN TIN WOODLAND ANIMALS]       [PLASTERS IN TIN CIRCUS PARADE]   
1     [PLASTERS IN TIN CIRCUS PARADE]    [PLASTERS IN TIN WOODLAND ANIMALS]   
2  [PLASTERS IN TIN WOODLAND ANIMALS]            [PLASTERS IN TIN SPACEBOY]   
3          [PLASTERS IN TIN SPACEBOY]    [PLASTERS IN TIN WOODLAND ANIMALS]   
4       [SET/6 RED SPOTTY PAPER CUPS]  [SET/20 RED RETROSPOT PAPER NAPKINS]   

   confidence  
0    0.597015  
1    0.606061  
2    0.611940  
3    0.759259  
4    0.740741  


**FP-Growth Algorithm**

In [ ]:
class ItemData :
  def __init__(self, item, sup_count):
    self.item = item
    self.sup_count = sup_count
    self.con_sup_cnt = 0

  def __str__(self):
     return str({
         "item" : self.item,
         "sup_count" : self.sup_count
     })

  def __cmp__(self,other):
    return self.count < other.count

class TreeNode :
  def __init__(self, data):
      self.parent = None
      self.children = {}
      self.next = None
      self.reference = None
      self.data = data

class ReferenceNode : 
  def __init__(self) :
    self.head = None
    self.tail = None
    self.sup_count = 0
    self.con_sup_cnt = 0
    self.item = None
  
  def __str__(self):
    return str({
        "item" : self.item,
        "sup_count" : self.sup_count,
        "con_sup_count" : self.con_sup_cnt
    })

  def __hash__(self):
    return hash(self.item)

  def __eq__(self, other):
    return (
    self.__class__ == other.__class__ and
            self.item == other.item )
  


In [ ]:
def generate_fp_tree(database, unique_items, min_sup_cnt):

  ############################# Part (1) ##################################################
  c_1 = [[item] for item in unique_items]
  itemset_df = calc_candidate_sup_cnt(database, c_1, 1)

  infrequent_rows = itemset_df[ itemset_df['support_cnt'] < min_sup_cnt ] # filter items with lower confidence and retrieve indices of them
  print(f'Infrequent Row : {infrequent_rows}\n')
  infrequent_items = set(row['itemset'][0] for index,row in infrequent_rows.iterrows()) ################################## TODO : Need to review ##########################################
  print( f'Infrequent Items : {infrequent_items}\n')
  
  #### Drop infrequent items ####
  itemset_df.drop(infrequent_rows.index , inplace=True)
  #### Sort according to support count ####
  itemset_df = itemset_df.sort_values(by = ['support_cnt'], ascending=False)
  print(f'Frequent single items : {itemset_df}\n\n')
 
  #### Remove infrequent items from every transaction and sort by support count in descending order ####
  database_node_model = []
  for transaction in database:
    transaction_prune = set(transaction)-infrequent_items # remove infrequent items from every transaction

    transaction_node_model = []
    for item in transaction_prune : 
      sup_cnt = itemset_df[itemset_df['key']==frozenset([item])]['support_cnt'].values[0]
      #### Create item object for each item in every transaction ###
      item_data = ItemData(item, sup_cnt)
      transaction_node_model.append(item_data)
    
    #### Sort item nodes in transaction accoring to support count (Descending order) ####
    transaction_node_model.sort(key = lambda c: c.sup_count, reverse=True) 
    database_node_model.append(transaction_node_model)

  ############################# Part (2) ##################################################
  
  #### Root Node : Null ####
  root_data = ItemData(item = None, sup_count = None)
  root_node = TreeNode(root_data) 
  #### Item chain maintain dictionary ####
  reference_keeper = {}
  
  ### Construct the model ####
  for transaction in database_node_model :
    current_root_locator = root_node
    for search_data in transaction:
      current_root_locator = traverse(current_root_locator, search_data, reference_keeper)

  return root_node, reference_keeper



def traverse(root_node, search_data, reference_keeper):
  current_root_locator = root_node
  if search_data.item not in current_root_locator.children:
    search_data.sup_count = 1
    search_node = TreeNode(search_data)
    search_node.parent = current_root_locator
    current_root_locator.children[search_data.item] = search_node
    current_root_locator = search_node

    if search_data.item not in reference_keeper :
      ##### Create reference for the new item ####
      reference_node = ReferenceNode()
      reference_node.head = search_node
      reference_node.tail = search_node
      reference_node.sup_count = search_data.sup_count
      reference_node.item = search_data.item
      reference_keeper[search_data.item] = reference_node
      search_node.reference = reference_node
    else :
      reference_node = reference_keeper[search_data.item]
      tail_node = reference_node.tail
      tail_node.next = search_node
      reference_node.tail = search_node
      reference_node.sup_count += search_data.sup_count
      reference_keeper[search_data.item] = reference_node
      search_node.reference = reference_node
  else :
    current_root_locator = current_root_locator.children[search_data.item]
    current_root_locator.data.sup_count += 1

    ##### Update Reference  ####
    reference_node = reference_keeper[search_data.item]
    reference_node.sup_count += 1
  return current_root_locator
  
def print_tree(tree_node) : 
  if(tree_node == None ) :
    return
  print(f'{tree_node.data}')
  children = tree_node.children
  if(len(children)==0) :
    return
  for child in children.values():
    print_tree(child)

def generate_fp_itemsets(root_node, reference_keeper, min_sup_cnt):

  ### sort references in descending order of support count ####
  reference_keeper = dict(sorted(reference_keeper.items(), key=lambda item: item[1].sup_count))

  all_itemsets = []
  for (item,reference) in reference_keeper.items() :
    if(reference.sup_count >= min_sup_cnt) :
      reference.con_sup_count = reference.sup_count
      generate_conditional_fp_itemsets(root_node, [] ,min_sup_cnt, reference, all_itemsets)
  print(f'Size of all itemsets : {len(all_itemsets)}')
  print(all_itemsets)
  # all_itemsets = set(all_itemsets)
  # print(len(all_itemsets))
      
   
def generate_conditional_fp_itemsets(root_node, conditional_itemset, min_sup_cnt, frequent_item_reference, all_itemsets):
   
  ##################### Update conditional support counts from bottom to top ###########################
  immediate_parent = frequent_item_reference.head.parent
  next_conditional_itemset = [frequent_item_reference.item, *conditional_itemset]
  all_itemsets.append(next_conditional_itemset)

  if(immediate_parent.data.item == None): # Root Node Check : Terminating condition
    return  

  con_next_reference_keeper = set()
  current_x_ref = frequent_item_reference.head
  
  ################ Reset conditional support count ##################
  # frequent_item_reference.con_sup_count = 0

  ##################### select prefix paths and update conditional support counts #######################
  while(current_x_ref != None) : 
    # current_x_ref.data.con_sup_cnt = 0
    current_y_ref = current_x_ref.parent

    #### Update conditional support counts through prefix paths from bottom to top ####
    while(current_y_ref.data.item != None): #### vertical movement until root ####
      con_next_reference_keeper.add(current_y_ref.reference)
      current_y_ref.reference.con_sup_cnt += 1
      # current_y_ref.data.con_sup_cnt += 1
      current_y_ref = current_y_ref.parent

    current_x_ref = current_x_ref.next   #### Horizontal movement ####
    
  ############### Remove low frequent nodes for the further process ############################
  in_frq_item_references = [reference for reference in con_next_reference_keeper if reference.con_sup_cnt < min_sup_cnt ]

  for inf_frq_item_ref in in_frq_item_references:
    inf_frq_item_ref.con_sup_cnt = 0
    con_next_reference_keeper.remove(inf_frq_item_ref)


  if len(con_next_reference_keeper)==0: # No more frequent ietm in the conditional tree :::  Terminating condition
    return 

  ############### Sort in ascending order of support count  ##################
  next_frequent_item_references = sorted(con_next_reference_keeper, key=lambda x: x.con_sup_cnt)
  for frq_item_ref in next_frequent_item_references:
    frq_item_ref.con_sup_cnt = 0

  for next_reference in next_frequent_item_references:
    generate_conditional_fp_itemsets(root_node, next_conditional_itemset, min_sup_cnt,next_reference, all_itemsets)
  return 
  


In [ ]:
########################################### Unit Testing ::: Small Dataset ###################################################################
(root_node, reference_keeper) = generate_fp_tree(database = database, unique_items = unique_items, min_sup_cnt = 2)
#### Print reference chain for each frequent items ####
print('Print Reference Chain for Each Frequent Items')
for (key,value) in reference_keeper.items() :
  print(f'Support Count of {value.head.data.item} : {value.sup_count}')
  temp = value.head
  while(temp != None) :
    print(temp.data)
    temp = temp.next
  print()

#### Generate itemsets using FP tree ####
generate_fp_itemsets(root_node, reference_keeper, min_sup_cnt)

Infrequent Row : Empty DataFrame
Columns: [key, item_cnt, itemset, support_cnt]
Index: []

Infrequent Items : set()

Frequent single items :              key item_cnt       itemset support_cnt
7        (BREAD)        1       [BREAD]          11
1       (COFFEE)        1      [COFFEE]           6
2          (TEA)        1         [TEA]           6
6      (BISCUIT)        1     [BISCUIT]           6
5   (CORNFLAKES)        1  [CORNFLAKES]           5
8         (MILK)        1        [MILK]           5
9        (MAGGI)        1       [MAGGI]           5
10       (SUGER)        1       [SUGER]           5
4    (BOURNVITA)        1   [BOURNVITA]           3
0         (COCK)        1        [COCK]           2
3          (JAM)        1         [JAM]           2


Print Reference Chain for Each Frequent Items
Support Count of BREAD : 11
{'item': 'BREAD', 'sup_count': 11}

Support Count of BISCUIT : 6
{'item': 'BISCUIT', 'sup_count': 4}
{'item': 'BISCUIT', 'sup_count': 1}
{'item': 'BISCUIT', 's

In [ ]:
########################################### Unit Testing ::: Large Dataset ###################################################################
(root_node, reference_keeper_large) = generate_fp_tree(database=database_large, unique_items=unique_items_large, min_sup_cnt = 35)
#### Print reference chain for each frequent items ####
print('Print Reference Chain for Each Frequent Items')
for (key,value) in reference_keeper_large.items() :
  print(f'Support Count of {value.head.data.item} : {value.sup_count}')
  temp = value.head
  while(temp != None) :
    print(temp.data)
    temp = temp.next
  print()

#### Generate itemsets using FP tree ####
generate_fp_itemsets(root_node, reference_keeper_large, 35)

Infrequent Row :                                     key item_cnt  \
0              (10 COLOUR SPACEBOY PEN)        1   
1          (12 COLOURED PARTY BALLOONS)        1   
2           (12 EGG HOUSE PAINTED WOOD)        1   
3     (12 MESSAGE CARDS WITH ENVELOPES)        1   
4       (12 PENCIL SMALL TUBE WOODLAND)        1   
...                                 ...      ...   
1557        (ZINC FOLKART SLEIGH BELLS)        1   
1558       (ZINC HERB GARDEN CONTAINER)        1   
1559      (ZINC METAL HEART DECORATION)        1   
1560   (ZINC T-LIGHT HOLDER STAR LARGE)        1   
1561  (ZINC T-LIGHT HOLDER STARS SMALL)        1   

                                itemset support_cnt  
0              [10 COLOUR SPACEBOY PEN]          12  
1          [12 COLOURED PARTY BALLOONS]           6  
2           [12 EGG HOUSE PAINTED WOOD]           1  
3     [12 MESSAGE CARDS WITH ENVELOPES]           2  
4       [12 PENCIL SMALL TUBE WOODLAND]           6  
...                               

**Comparison with state-of-the-art techniques**

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

#Let's transform the list, with one-hot encoding
from mlxtend.preprocessing import TransactionEncoder
a = TransactionEncoder()
a_data = a.fit(database).transform(database)
df = pd.DataFrame(a_data,columns=a.columns_)
df = df.replace(False,0)
df


df = apriori(df, min_support = 0.1, use_colnames = True)
print(f'Shape of frequent itemsets : {df}')
print(f'Frequent itemsets from apriori \n{df.head()}')
print(f'Frequent itemsets from apriori approach \n{df.tail()}')

df_ar = association_rules(df, metric = "confidence", min_threshold = 0.5)
print(df_ar)
print(f'\n\nShape of association rules : {df_ar.shape}')
print(f'Association rules(Head) \n{df_ar.head()}')
print(f'Association rules(Tail) \n{df_ar.tail()}')

Shape of frequent itemsets :      support                itemsets
0   0.352941               (BISCUIT)
1   0.176471             (BOURNVITA)
2   0.647059                 (BREAD)
3   0.117647                  (COCK)
4   0.352941                (COFFEE)
5   0.294118            (CORNFLAKES)
6   0.117647                   (JAM)
7   0.294118                 (MAGGI)
8   0.294118                  (MILK)
9   0.294118                 (SUGER)
10  0.352941                   (TEA)
11  0.235294        (BISCUIT, BREAD)
12  0.117647   (BISCUIT, CORNFLAKES)
13  0.117647        (MAGGI, BISCUIT)
14  0.117647         (MILK, BISCUIT)
15  0.117647          (TEA, BISCUIT)
16  0.117647      (BOURNVITA, BREAD)
17  0.117647      (SUGER, BOURNVITA)
18  0.117647         (COFFEE, BREAD)
19  0.117647            (JAM, BREAD)
20  0.176471          (MAGGI, BREAD)
21  0.235294           (MILK, BREAD)
22  0.176471          (SUGER, BREAD)
23  0.176471            (TEA, BREAD)
24  0.117647          (COCK, COFFEE)
25  0.176